In [ ]:
import json

with open('plant-database.json') as f:
    plants = json.loads(f.read())
print(len(plants))

In [ ]:
import asyncio
from langchain_community.document_loaders import WikipediaLoader

async def load_plant(plant):
    try:
        docs = await WikipediaLoader(query=plant['latinName'], load_max_docs=1, lang='de').aload()
        if len(docs) == 0:
            docs = await WikipediaLoader(query=plant['name'], load_max_docs=1, lang='de').aload()
        if len(docs) == 0:
            return (plant['latinName'], None, None)
        return (plant['latinName'], docs[0].page_content, docs[0].metadata['source'])
    except:
        return (plant['latinName'], None, None)

async def load_plants(plants_queue, result_queue):
    while not plants_queue.empty():
        plant = await plants_queue.get()
        res = await load_plant(plant)
        await result_queue.put(res)
        plants_queue.task_done()
    return True

In [ ]:
plants_queue = asyncio.Queue(0)
result_queue = asyncio.Queue(0)

for plant in plants:
    plants_queue.put_nowait(plant)

await asyncio.gather(*[ load_plants(plants_queue, result_queue) for _ in range(10)  ])

plant_documents = []
while not result_queue.empty():
    plant_documents.append(result_queue.get_nowait())
len(plant_documents)

In [ ]:
import pandas as pd

plants_df = pd.DataFrame(plant_documents, columns=['latinName', 'document', 'source'])
plants_df

In [ ]:
mising = set(plants_df[plants_df['document'].isna()]['latinName'].values)

missing_plants = [ plant for plant in plants if plant['latinName'] in mising ]
len(missing_plants)

In [ ]:
plants_queue = asyncio.Queue(0)
result_queue = asyncio.Queue(0)

for plant in missing_plants:
    plants_queue.put_nowait(plant)

await asyncio.gather(*[ load_plants(plants_queue, result_queue) for _ in range(10) ])

plant_documents2 = []
while not result_queue.empty():
    plant_documents2.append(result_queue.get_nowait())
len(plant_documents2)

In [ ]:
plants_df2 = pd.DataFrame(plant_documents2, columns=['latinName', 'document', 'source'])
plants_df2

In [ ]:
len(plants_df2[plants_df2['document'].isna()]['latinName'].values)

In [ ]:
plants_df = pd.concat([plants_df.dropna(), plants_df2.dropna()])
plants_df

In [1]:
import re

def cleanup(value):
    value = value.replace('\n', ' ')
    value = re.sub(r'=+', ' ', value)
    value = re.sub(r' +', ' ', value)
    value = re.sub(r';', '.', value)
    return value

plants_df['document'] = plants_df['document'].apply(cleanup)
plants_df.to_csv('plants.tsv', sep='\t', index=False)

In [2]:
from langchain_community.document_loaders import DataFrameLoader

loader = DataFrameLoader(plants_df, page_content_column='document')
docs = loader.load()
len(docs)

2515

In [4]:
from langchain_text_splitters import SentenceTransformersTokenTextSplitter

text_splitter = SentenceTransformersTokenTextSplitter(model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
split_docs = text_splitter.split_documents(docs)
len(split_docs)

33861

In [10]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS

embeddings = OllamaEmbeddings(model='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', base_url='http://localhost:11434', query_instruction='', embed_instruction='')

In [13]:
%%time
db = FAISS.from_documents(split_docs, embeddings)

CPU times: user 2min 43s, sys: 49.7 s, total: 3min 33s
Wall time: 1h 1min 10s


In [14]:
db.save_local('plantsdb-sentence_transformer-l12')

In [30]:
question = 'Trockenliebende genügsame Pflanze mit gelben Blütenblättern'
docs = db.similarity_search_with_score(question, k=10)

docs

[(Document(page_content='„bereift“. Die zwittrigen und duftenden Blüten sind gelbe, halbkugelig-glockige, nektarführende Scheibenblumen. Sie finden sich in bis zu dreißigblütigen und hängenden, traubigen Blütenständen, die sich endständig an Kurztrieben befinden. Die Blüten besitzen jeweils bootförmige, sechs grünlich-gelbe Kelchblätter in zwei Kreisen, drei äußere kleinere und drei größere innere, sowie sechs, etwas kleinere, gelbe Kronblätter mit kleinen, basalen', metadata={'latinName': 'Berberis vulgaris', 'source': 'https://de.wikipedia.org/wiki/Gew%C3%B6hnliche_Berberitze'}),
  4.480606),
 (Document(page_content='mal so lang wie ihre Deckblätter. Die relativ kleinen, zwittrigen Blüten besitzen eine doppelte Blütenhülle. Die Blütenkrone ist blassblau und dunkler (gestreift) geadert und besitzt einen gelblichen Schlund. Der Fruchtstiel ist zwei- bis dreimal so lang wie der Blütenkelch. Die sehr tief spitzwinklig ausgerandete, stark abgeflachte Kapselfrucht ist so lang wie die Kelch